## LangChain is a framework for developing applications powered by language models.

In [ ]:
# !pip install langchain
# !pip install openai
# !pip install wikipedia

## Building a language model application

In [1]:
from langchain.llms import OpenAI
import os

In [2]:
# SET YOUR API KEY
os.environ['OPENAI_API_KEY']= 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

### LangChain provides a generic interface for many different LLMs. Most of them work via their API but you can also run local models.

In [3]:
# temperature determines the randomness in the output 

llm = OpenAI(temperature=0.9)

### Lets ask some questions ❔❔❔

In [6]:
text = 'What are the five vacation destinations for someone who likes to eat pasta?'
print(llm(text))



1. Rome, Italy
2. Bologna, Italy 
3. Naples, Italy 
4. Sicily, Italy 
5. Venice, Italy


### Prompt Templates: Manage prompts for LLMs

In [8]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=['food'],
    template='What are the five vacation destinations for someone who likes to eat {food}?')

In [10]:
print(prompt.format(food='dessert'))

What are the five vacation destinations for someone who likes to eat dessert?


In [11]:
print(llm(prompt.format(food='dessert')))



1. Paris, France - Home to some of the best pastries and desserts in the world!
2. Rome, Italy - Try some of the delicious gelato treats here.
3. Tokyo, Japan - Indulge in some traditional Japanese desserts like mochi and dango.
4. Bangkok, Thailand - Enjoy exotic and flavorful sweet treats like mango sticky rice and tub tim grob.
5. New York City, USA - From the classic New York cheesecake to trendy treat shops, this city is sure to satisfy any sweet tooth.


### LangChain faciliates prompt management and optimization.

Normally, when you use an LLM in an application, you are not sending user input directly to the LLM. Instead, you need to take the user input and construct a prompt, and only then send that to the LLM.

In [7]:
print(llm("Can Barack Obama have a conversation with George Washington?"))



No, Barack Obama cannot have a conversation with George Washington because George Washington died in 1799.


#### A better prompt is this:

<code>
prompt = """Question: Can Barack Obama have a conversation with George Washington?
<br>
<br>
Let's think step by step.

Answer: """

llm(prompt)
</code>

#### This can be achieved with PromptTemplate

In [8]:
prompt = PromptTemplate(
    template="""Question: {question}?

    Let's think step by step.

    Answer: """,
    
    input_variables=['question']
)

In [10]:
print(prompt.format(question='Can Barack Obama have a conversation with George Washington'))

Question: Can Barack Obama have a conversation with George Washington?

    Let's think step by step.

    Answer: 


In [5]:
from langchain import PromptTemplate

prompt = PromptTemplate(
    input_variables=['adjective','subject'], 
    template="Write a 2 line {adjective} song about {subject}"
)

print(prompt.format(adjective='sad', subject='ducks'))

Write a 2 line sad song about ducks


In [15]:
print(llm(prompt.format(adjective='sad', subject='duck')))



Verse 1:
I waddle along, with my head held low 
Though life has been hard, I try not to show. 

Chorus:
No one to comfort me, no one to care 
My heart is so heavy, so full of despair.


## Chains : Combine LLMs and prompts in a multistep workflow


In [11]:
prompt = PromptTemplate(
    template="""Question: {question}?

    Let's think step by step.

    Answer: """,
    
    input_variables=['question']
)

print(prompt.format(question='Can Barack Obama have a conversation with George Washington'))

Question: Can Barack Obama have a conversation with George Washington?

    Let's think step by step.

    Answer: 


In [13]:
from langchain import LLMChain

llm = OpenAI(temperature=0.9)
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Can Barack Obama have a conversation with George Washington"

In [14]:
print(llm_chain.run(question))


No, it is not possible for Barack Obama to have a conversation with George Washington since George Washington passed away in 1799.


### Agents : Dynamically call chains based on user input

#### Agents and Tools
Agents involve an LLM making decisions about which actions to take, taking that action, seeing an observation, and repeating that until done.

When used correctly agents can be extremely powerful. In order to load agents, you should understand the following concepts:

* Tool: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains.
* LLM: The language model powering the agent.
* Agent: The agent to use.

In [18]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain import OpenAI



# LLM
llm = OpenAI(temperature=0)

# tools
tools = load_tools(tool_names=['wikipedia', 'llm-math'], llm=llm)

# Agent
agent = initialize_agent(tools=tools, llm=llm, agent="zero-shot-react-description", verbose=True)

In [19]:
agent.run("In what year was the film Departed with Leopnardo Dicaprio released? What is this year raised to the 0.43 power?")



> Entering new AgentExecutor chain...
 I need to find out the year the film was released and then use the calculator to calculate the power.
Action: Wikipedia
Action Input: Departed with Leonardo Dicaprio
Observation: Page: Leonardo DiCaprio
Summary: Leonardo Wilhelm DiCaprio (, ; Italian: [diˈkaːprjo]; born November 11, 1974) is an American actor and film producer. Known for his work in biographical and period films, he is the recipient of numerous accolades, including an Academy Award, a British Academy Film Award and three Golden Globe Awards. As of 2019, his films have grossed over $7.2 billion worldwide, and he has been placed eight times in annual rankings of the world's highest-paid actors.
Born in Los Angeles, DiCaprio began his career in the late 1980s by appearing in television commercials. In the early 1990s, he had recurring roles in various television shows, such as the sitcom Parenthood, and had his first major film part as author Tobias Wolff in This Boy's Life (1993).

'2006 raised to the 0.43 power is approximately 1.7.'

## Memory
Add state to Chains and Agents.

Memory is the concept of persisting state between calls of a chain/agent. 



In [22]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0)

conversation = ConversationChain(llm=llm, verbose=True)

In [23]:
conversation.predict(input='Hello !')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello !
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

In [24]:
conversation.predict(input='Can we talk about AI')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello !
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: Can we talk about AI
AI:

> Finished chain.


' Absolutely! What would you like to know about AI?'

In [26]:
conversation.predict(input='I like to know about Machine Learning')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello !
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: Can we talk about AI
AI:  Absolutely! What would you like to know about AI?
Human: I like to know about Machine Learning
AI:

> Finished chain.


' Sure! Machine Learning is a subset of Artificial Intelligence that focuses on the development of computer programs that can access data and use it to learn for themselves. It is a type of AI that allows machines to learn from data without being explicitly programmed.'